In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/assignment-3/data/Task1/dev.csv
/kaggle/input/assignment-3/data/Task1/sample_test.csv
/kaggle/input/assignment-3/data/Task1/sample_demo.csv
/kaggle/input/assignment-3/data/Task1/train.csv
/kaggle/input/assignment-3/data/Task2/train_data.json
/kaggle/input/assignment-3/data/Task2/validation_data.json
/kaggle/input/assignment-3/data/Task2/test.json
/kaggle/input/assignment-3/data/A3_task1_data_files/dev.csv
/kaggle/input/assignment-3/data/A3_task1_data_files/sample_test.csv
/kaggle/input/assignment-3/data/A3_task1_data_files/sample_demo.csv
/kaggle/input/assignment-3/data/A3_task1_data_files/train.csv
/kaggle/input/assignment-3/A3_task1_data_files/dev.csv
/kaggle/input/assignment-3/A3_task1_data_files/sample_test.csv
/kaggle/input/assignment-3/A3_task1_data_files/sample_demo.csv
/kaggle/input/assignment-3/A3_task1_data_files/train.csv
/kaggle/input/assignment-3/assignment-3/dev.csv
/kaggle/input/assignment-3/assignment-3/sample_test.csv
/kaggle/input/assignment-3/assignment

In [2]:
! pip install -U sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 4.2 MB/s eta 0:00:00


In [3]:
from sentence_transformers import SentenceTransformer,util
import torch
import scipy
from scipy.stats import pearsonr

In [4]:
validation_data = pd.read_csv("/kaggle/input/assignment-3/assignment-3/dev.csv",sep="\t")
validation_data

,score,sentence1,sentence2
0,5.00,A man with a hard hat is dancing.,A man wearing a hard hat is dancing.
1,4.75,A young child is riding a horse.,A child is riding a horse.
2,5.00,A man is feeding a mouse to a snake.,The man is feeding a mouse to the snake.
3,2.40,A woman is playing the guitar.,A man is playing guitar.
4,2.75,A woman is playing the flute.,A man is playing a flute.
...,...,...,...
1465,2.00,Scientists prove there is water on Mars,Has Nasa discovered water on Mars?
1466,0.00,Pranab stresses need to strive for peace by na...,WTO: India regrets action of developed nations
1467,2.00,Volkswagen skids into red in wake of pollution...,"Volkswagen's ""gesture of goodwill"" to diesel o..."
1468,0.00,Obama is right: Africa deserves better leadership,Obama waiting for midterm to name attorney gen...


In [5]:
# Finding the number of sentences with na entries in the sentence1 column
print(validation_data['sentence1'].isna().sum())

0


In [6]:
# Finding the number of sentences with na entries in the sentence2 column
print(validation_data['sentence2'].isna().sum())

2


In [7]:
validation_data.dropna(inplace=True)
print(validation_data['sentence2'].isna().sum())

0


In [8]:
validation_data.shape

(1468, 3)

In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [10]:
model = SentenceTransformer("all-MiniLM-L6-v2")
model.to(device)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [11]:
sentence_set_1 = validation_data['sentence1'].to_numpy()
sentence_set_2 = validation_data['sentence2'].to_numpy()
sentence_set_1.shape,sentence_set_2.shape

((1468,), (1468,))

In [12]:
encode_sentence_set_1 = model.encode(sentence_set_1)
encode_sentence_set_2 = model.encode(sentence_set_2)


Batches:   0%|          | 0/46 [00:00<?, ?it/s]

Batches:   0%|          | 0/46 [00:00<?, ?it/s]

In [13]:
cosine_similarities = util.cos_sim(encode_sentence_set_1,encode_sentence_set_2).tolist()


In [14]:
len(encode_sentence_set_1[0])

384

In [15]:
predicted_scores = []
pairwise_pearson=[]
for i in range(len(cosine_similarities)):
    cos_score = (cosine_similarities[i][i]+1)/2
    pairwise_pearson.append(pearsonr(encode_sentence_set_1[i],encode_sentence_set_2[i])[0])
    predicted_scores.append(cos_score*5)
    
    
data={
    'actual_score':validation_data['score'],
    'predicted_score':predicted_scores,
    'sentence1':sentence_set_1,
    'sentence2':sentence_set_2
}
final_dataframe = pd.DataFrame(data)
final_dataframe.to_csv("task_1_B.csv")
final_dataframe

,actual_score,predicted_score,sentence1,sentence2
0,5.00,4.983430,A man with a hard hat is dancing.,A man wearing a hard hat is dancing.
1,4.75,4.884907,A young child is riding a horse.,A child is riding a horse.
2,5.00,4.950253,A man is feeding a mouse to a snake.,The man is feeding a mouse to the snake.
3,2.40,4.117708,A woman is playing the guitar.,A man is playing guitar.
4,2.75,4.365570,A woman is playing the flute.,A man is playing a flute.
...,...,...,...,...
1465,2.00,4.635375,Scientists prove there is water on Mars,Has Nasa discovered water on Mars?
1466,0.00,3.395640,Pranab stresses need to strive for peace by na...,WTO: India regrets action of developed nations
1467,2.00,3.595423,Volkswagen skids into red in wake of pollution...,"Volkswagen's ""gesture of goodwill"" to diesel o..."
1468,0.00,3.256864,Obama is right: Africa deserves better leadership,Obama waiting for midterm to name attorney gen...


In [16]:
score_val = validation_data['score']
score_pred_val = final_dataframe['predicted_score']

print(f"Pearson's Coeffecient = {pearsonr(score_val,score_pred_val)[0]}")

Pearson's Coeffecient = 0.8631423775259331
